In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Grounding in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grounding.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grounding.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/grounding/intro-grounding.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.11

## Overview

[Grounding in Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/grounding/overview) lets you use language models (e.g., [`text-bison` and `chat-bison`](https://cloud.google.com/vertex-ai/docs/generative-ai/language-model-overview)) to generate content grounded in your own documents and data. This capability lets the model access information at runtime that goes beyond its training data. By grounding model responses in Google Search results or data stores within [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction), LLMs that are grounded in data can produce more accurate, up-to-date, and relevant responses.

Grounding provides the following benefits:

- Reduces model hallucinations (instances where the model generates content that isn't factual)
- Anchors model responses to specific information, documents, and data sources
- Enhances the trustworthiness, accuracy, and applicability of the generated content

In the context of grounding in Vertex AI, you can configure two different sources of grounding:

1. Google Search results for data that is publicly available and indexed
1. [Data stores in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest), which can include your own data in the form of website data, unstructured data, or structured data

### Objective

In this tutorial, you learn how to:

- Generate LLM text and chat model responses grounded in Google Search results
- Compare the results of ungrounded LLM responses with grounded LLM responses
- Create and use a data store in Vertex AI Search to ground responses in custom documents and data
- Generate LLM text and chat model responses grounded in Vertex AI Search results
- Use the asynchronous text and chat models APIs with grounding

This tutorial uses the following Google Cloud AI services and resources:

- Vertex AI
- Vertex AI Search and Conversation

The steps performed include:

- Configuring the LLM and prompt for various examples
- Sending example prompts to generative text and chat models in Vertex AI
- Setting up a data store in Vertex AI Search with your own data
- Sending example prompts with various levels of grounding (no grounding, web grounding, data store grounding)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. Enable the [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and [Vertex AI Search and Conversation API](https://console.cloud.google.com/flows/enableapi?apiid=discoveryengine.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

### Installation

Install the following packages required to execute this notebook.

In [2]:
!pip install --upgrade --quiet google-cloud-aiplatform==1.38.0

Restart the kernel after installing packages:

In [3]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Configure your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [4]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}

# Set the project ID
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Configure your region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [5]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [6]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [7]:
import vertexai
from vertexai.language_models import TextGenerationModel, ChatModel, GroundingSource

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project:

In [8]:
vertexai.init(project=PROJECT_ID, location=REGION)

Initialize the generative text and chat models from Vertex AI:

In [9]:
text_model = TextGenerationModel.from_pretrained("text-bison@001")
chat_model = ChatModel.from_pretrained("chat-bison@001")

## Example: Grounding with Google Search results

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search. You'll ask a question about a recent hardware release from the Google Store.

In [10]:
PROMPT = (
    "What are the price, available colors, and storage size options of a Pixel Tablet?"
)

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [11]:
response = text_model.predict(PROMPT)

response, response.grounding_metadata

(The Pixel Tablet will be available in three colors: Stormy Black, Kinda Coral, and Sorta Seafoam. It will have 128GB and 256GB storage options. The price has not yet been announced.,
 GroundingMetadata(citations=[], search_queries=[]))

### Text generation grounded in Google Search results

Now you can add the `grounding_source` keyword arg with a grounding source of `GroundingSource.WebSearch()` to instruct the LLM to first perform a Google Search with the prompt, then construct an answer based on the web search results:

In [12]:
grounding_source = GroundingSource.WebSearch()

response = text_model.predict(
    PROMPT,
    grounding_source=grounding_source,
)

response, response.grounding_metadata

(The Pixel Tablet starts at $499 in the US, £599 in the UK, €679 throughout selected European regions, and CAD $699 in Canada. The tablet comes in three colors: Porcelain (white), Rose (pink), and Hazel (gray). The Pixel Tablet is available in two storage size options: 8GB/128GB and 8GB/256GB.,
 GroundingMetadata(citations=[GroundingCitation(start_index=0, end_index=128, url='https://www.androidauthority.com/google-pixel-tablet-3163922/', title=None, license=None, publication_date=None), GroundingCitation(start_index=129, end_index=212, url='https://www.androidpolice.com/google-pixel-tablet/', title=None, license=None, publication_date=None), GroundingCitation(start_index=213, end_index=296, url='https://priceinall.com/product/google-pixel-tablet/', title=None, license=None, publication_date=None)], search_queries=['Pixel Tablet price, colors, and storage size options?']))

Note that the response without grounding only has outdated information from the LLM about the potential release date of the Pixel tablet. Whereas the response that was grounded in web search results contains the most up to date information from web search results that are returned as part of the LLM with grounding request.

## Example: Grounding with custom documents and data

In this example, you'll compare LLM responses with no grounding with responses that are grounded in the [results of a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest). You'll ask a question about a GoogleSQL query to create an [object table in BigQuery](https://cloud.google.com/bigquery/docs/object-table-introduction).

### Creating a data store in Vertex AI Search

Follow the steps in the [Vertex AI Search getting started documentation](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app_for_website_data) to create a data store in Vertex AI Search with sample data. In this example, you'll use a website-based data store that contains content from the Google Cloud website, including documentation.

Once you've created a data store, obtain the Data Store ID and input it below.

In [13]:
DATA_STORE_ID = "your-data-store-id_1234567890123"  # Replace this with your data store ID from Vertex AI Search
DATA_STORE_REGION = "global"

Now you can ask a question about object tables in BigQuery and when to use them:

In [14]:
PROMPT = "When should I use an object table in BigQuery? And how does it store data?"

### Text generation without grounding

Make a prediction request to the LLM with no grounding:

In [15]:
response = text_model.predict(PROMPT)

response.grounding_metadata, response

(GroundingMetadata(citations=[], search_queries=[]),
 Object tables are a type of table in BigQuery that can store data in a variety of formats, including JSON, Avro, Parquet, and ORC. They are useful when you need to store data that is not natively supported by BigQuery, such as images or videos.

Object tables are stored in a different way than regular BigQuery tables. Regular BigQuery tables are stored in columnar format, which means that the data is stored in rows, with each column being a separate data field. Object tables, on the other hand, are stored in row-oriented format, which means that the data is stored in rows)

### Text generation grounded in Vertex AI Search results

Now we can add the `grounding_source` keyword arg with a grounding source of `GroundingSource.VertexAISearch()` to instruct the LLM to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [16]:
grounding_source = GroundingSource.VertexAISearch(
    data_store_id=DATA_STORE_ID, location=DATA_STORE_REGION
)

response = text_model.predict(
    PROMPT,
    grounding_source=grounding_source,
)

response.grounding_metadata, response

(GroundingMetadata(citations=[GroundingCitation(start_index=0, end_index=65, url='https://cloud.google.com/bigquery/docs/object-table-introduction', title=None, license=None, publication_date=None)], search_queries=['When should I use an object table in BigQuery?']),
 Object tables let you analyze unstructured data in Cloud Storage. You can perform analysis with remote functions or perform inference by using BigQuery ML, and you can also use object tables as input to BigQuery ML models.

Object tables store data in Cloud Storage as files. The files can be of any type, and they can be stored in any location in Cloud Storage.)

Note that the response without grounding only has limited information from the LLM about object tables in BigQuery that might not be accurate. Whereas the response that was grounded in Vertex AI Search results contains the most up to date information about object tables based on documents returned from the Google Cloud documentation about BigQuery.

## Example: Grounded chat responses

You can also use grounding when working with chat models in Vertex AI. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a Google Search and a data store in Vertex AI Search.

You'll ask a question about Vertex AI and a follow up question about managed datasets in Vertex AI:

In [17]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use"

### Chat session without grounding

Start a chat session and send messages to the LLM with no grounding:

In [18]:
chat = chat_model.start_chat()

response = chat.send_message(PROMPT)
print(response.grounding_metadata)
print(response.text)

response = chat.send_message(PROMPT_FOLLOWUP)
print(response.grounding_metadata)
print(response.text)

GroundingMetadata(citations=[], search_queries=[])
Managed datasets in Vertex AI Managed datasets are a fully managed service that provides a simple and scalable way to store and manage your data. They are built on top of Google Cloud Storage and BigQuery, and they provide a number of features that make it easy to work with your data, including:

* Automatic schema discovery and validation
* Automatic partitioning and replication
* Built-in support for machine learning models
* Integration with other Google Cloud services

Managed datasets are a great option for organizations that want to store and manage their data in a scalable and cost-effective way. They are also a good choice for organizations that want to use machine learning models to analyze their data.
GroundingMetadata(citations=[], search_queries=[])
Managed datasets can store any type of data, including structured, semi-structured, and unstructured data. They can also store data from a variety of sources, including Google C

### Chat session grounded in Google Search results

Now you can add the `grounding_source` keyword arg with a grounding source of `GroundingSource.WebSearch()` to instruct the chat model to first perform a Google Search with the prompt, then construct an answer based on the web search results:

In [19]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.WebSearch()

response = chat.send_message(
    PROMPT,
    grounding_source=grounding_source,
)

response = chat.send_message(PROMPT)
print(response.grounding_metadata)
print(response.text)

response = chat.send_message(PROMPT_FOLLOWUP)
print(response.grounding_metadata)
print(response.text)

GroundingMetadata(citations=[], search_queries=[])
Managed datasets in Vertex AI are a way to store and manage your data for machine learning (ML) models. They are fully managed, which means that Google takes care of the underlying infrastructure and storage, so you can focus on building your models. Managed datasets also support versioning, so you can track changes to your data over time.
GroundingMetadata(citations=[], search_queries=[])
You can use any type of data that can be stored in a Google Cloud Storage bucket, including tabular data, image data, and text data.


### Chat session grounded in Vertex AI Search results

Now you can add the `grounding_source` keyword arg with a grounding source of `GroundingSource.VertexAISearch()` to instruct the chat model to first perform a search within your custom data store, then construct an answer based on the relevant documents:

In [20]:
chat = chat_model.start_chat()
grounding_source = GroundingSource.VertexAISearch(
    data_store_id=DATA_STORE_ID, location=DATA_STORE_REGION
)

response = chat.send_message(
    PROMPT,
    grounding_source=grounding_source,
)

response = chat.send_message(PROMPT)
print(response.grounding_metadata)
print(response.text)

response = chat.send_message(PROMPT_FOLLOWUP)
print(response.grounding_metadata)
print(response.text)

GroundingMetadata(citations=[], search_queries=[])
Managed datasets in Vertex AI are a way to store and manage your data for machine learning models. They are fully managed, which means that Google takes care of the underlying infrastructure and storage, so you can focus on building your models. Managed datasets also support versioning, so you can track changes to your data over time.
GroundingMetadata(citations=[], search_queries=[])
You can use any type of data that can be stored in a Google Cloud Storage bucket, including tabular data, image data, and text data.


## Example: Grounded async text and chat responses

You can also use grounding in Vertex AI when working with the asynchronous APIs for the text and chat models. In this example, you'll compare LLM responses with no grounding with responses that are grounded in the results of a data store in Vertex AI Search.

You'll ask a question about different services available in Google Cloud.

In [21]:
PROMPT = "What are the different types of databases available in Google Cloud?"

### Async text generation grounded in Google Search results

In [22]:
grounding_souce = GroundingSource.WebSearch()

response = await text_model.predict_async(
    PROMPT,
    grounding_source=grounding_souce,
)

response.grounding_metadata, response

(GroundingMetadata(citations=[GroundingCitation(start_index=134, end_index=172, url='https://cloud.google.com/blog/topics/developers-practitioners/your-google-cloud-database-options-explained', title=None, license=None, publication_date=None)], search_queries=['What are the different types of databases available in Google Cloud?']),
 Google Cloud Database offers fully managed solutions both for relational and non relational databases with high security features. · Relational : Cloud SQL and AlloyDB. · Non-relational : Cloud Spanner, Cloud Bigtable, Cloud Firestore, Cloud Datastore, Cloud Memorystore for Redis, Cloud Memorystore for Memcached, Cloud SQL for PostgreSQL, Cloud SQL for MySQL, Cloud SQL for SQL Server.)

### Async text generation grounded in Vertex AI Search results

In [23]:
grounding_souce = GroundingSource.VertexAISearch(
    data_store_id=DATA_STORE_ID, location=DATA_STORE_REGION
)

response = await text_model.predict_async(
    PROMPT,
    grounding_source=grounding_souce,
)

response.grounding_metadata, response

(GroundingMetadata(citations=[], search_queries=['What are the different types of databases available in Google Cloud?']),
 Google Cloud offers a variety of databases to meet the needs of your business. You can choose from relational databases, NoSQL databases, in-memory databases, and document databases.

Relational databases are the most common type of database. They store data in tables, which are organized into rows and columns. Relational databases are good for storing structured data, such as customer records or product information.

NoSQL databases are a newer type of database that are not based on the relational model. NoSQL databases are good for storing unstructured data, such as social media data or sensor data.

In-memory databases store data in RAM)

### Async chat session grounded in Google Search results

In [24]:
chat = chat_model.start_chat()

grounding_source = GroundingSource.WebSearch()
response = await chat.send_message_async(
    PROMPT,
    grounding_source=grounding_source,
)

response.grounding_metadata, response

(GroundingMetadata(citations=[], search_queries=['What are the different types of databases available in Google Cloud?']),
 Google Cloud offers a wide range of databases to meet the needs of your business. These include relational databases, NoSQL databases, and in-memory databases.

Relational databases are designed to store data in tables, with each row representing a record and each column representing a field. They are well-suited for applications that require structured data, such as customer relationship management (CRM) systems and enterprise resource planning (ERP) systems.

NoSQL databases are designed to store data in a more flexible way than relational databases. They do not require data to be stored in tables, and they can support different types of data, such as text, images, and videos. NoSQL databases are well-suited for applications that require high performance and scalability, such as web applications and mobile applications.

In-memory databases store data in RAM rat

### Async chat session grounded in Vertex AI Search results

In [25]:
chat = chat_model.start_chat()

grounding_source = GroundingSource.VertexAISearch(
    data_store_id=DATA_STORE_ID, location=DATA_STORE_REGION
)
response = await chat.send_message_async(
    PROMPT,
    grounding_source=grounding_source,
)

response.grounding_metadata, response

(GroundingMetadata(citations=[], search_queries=['What are the different types of databases available in Google Cloud?']),
 Google Cloud offers a wide range of database options to meet the needs of your business. These include relational databases, NoSQL databases, in-memory databases, and document databases.)

## Cleaning up

To avoid incurring charges to your Google Cloud account for the resources used in this notebook, follow these steps:

1. To avoid unnecessary Google Cloud charges, use the [Google Cloud console](https://console.cloud.google.com/) to delete your project if you do not need it. Learn more in the Google Cloud documentation for [managing and deleting your project](https://cloud.google.com/resource-manager/docs/creating-managing-projects).
1. If you used an existing Google Cloud project, delete the resources you created to avoid incurring charges to your account. For more information, refer to the documentation to [Delete data from a data store in Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores), then delete your data store.
1. Disable the [Vertex AI Search and Conversation API](https://pantheon.corp.google.com/apis/api/discoveryengine.googleapis.com) and [Vertex AI API](https://pantheon.corp.google.com/apis/api/aiplatform.googleapis.com) in the Google Cloud Console.